In [1]:
import pandas as pd
import numpy as np
import rasterio
from rasterio.transform import from_origin

csv_file = r"C:\Users\Jan\OneDrive\Uni\3. Semester\Seminar Urban Computing\ssep_open.csv"
value_column = "ssep1"
output_tif = "../geoserver_data/data/ssep.tif"
resolution = 100

df = pd.read_csv(csv_file)

if not {"geox", "geoy", value_column}.issubset(df.columns):
    raise ValueError(f"CSV muss die Spalten 'geox', 'geoy' und '{value_column}' enthalten.")

xmin, xmax = df["geox"].min(), df["geox"].max()
ymin, ymax = df["geoy"].min(), df["geoy"].max()

width = int((xmax - xmin) / resolution) + 1
height = int((ymax - ymin) / resolution) + 1

raster = np.full((height, width), np.nan)

transform = from_origin(xmin, ymax, resolution, resolution)

for _, row in df.iterrows():
    col = int((row["geox"] - xmin) / resolution)
    row_r = int((ymax - row["geoy"]) / resolution)
    if 0 <= row_r < height and 0 <= col < width:
        raster[row_r, col] = row[value_column]

with rasterio.open(
    output_tif,
    "w",
    driver="GTiff",
    height=height,
    width=width,
    count=1,
    dtype=raster.dtype,
    crs="EPSG:2056",
    transform=transform,
    nodata=np.nan
) as dst:
    dst.write(raster, 1)

print(f"GeoTIFF saved: {output_tif}")


GeoTIFF saved: ssep.tif
